# AWS CodePipeline を使って ML CI/CD パイプラインを作成する

このノートブックは、AWS CodePipeline を使って以下のような CI/CD パイプライン全体を作ります。

- AWS CodeCommit へのファイル変更でパイプラインを実行開始
- AWS Step Functions で作成した、データ準備、モデル学習、モデル評価、後処理のパイプラインを実行
- モデルデプロイ可否を手動承認
- 承認されたらモデルを Amazon SageMaker 推論エンドポイントにデプロイ

**このノートブックでは緩めの権限を持つ IAM Polycy と IAM Role を使用していますが、本番環境では最小限の権限を持つ Policy, Role を作成・使用してください。[IAM Access Analyzer](https://docs.aws.amazon.com/ja_jp/IAM/latest/UserGuide/access-analyzer-policy-generation.html) は、必要な権限のみを持つ Policy の作成をサポートします。**

<img src='architecture.png'>

## このサンプルのファイル構成

```
root
|- docker: 後処理 Lambda 用ファイル一式
|- git-repo-codes: CodeCommit に push するファイル一式。CodeBuild で使用
|- Create-ML-Model-CICD-Pipeline.ipynb: このノートブック。上記アーキテクチャを作成する
|- Dockerfile: ベースコンテナイメージ作成用
```

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AWS-CodePipeline-を使って-ML-CI/CD-パイプラインを作成する" data-toc-modified-id="AWS-CodePipeline-を使って-ML-CI/CD-パイプラインを作成する-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AWS CodePipeline を使って ML CI/CD パイプラインを作成する</a></span><ul class="toc-item"><li><span><a href="#このサンプルのファイル構成" data-toc-modified-id="このサンプルのファイル構成-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>このサンプルのファイル構成</a></span></li><li><span><a href="#環境のセットアップ" data-toc-modified-id="環境のセットアップ-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>環境のセットアップ</a></span><ul class="toc-item"><li><span><a href="#必要なモジュールのインポート" data-toc-modified-id="必要なモジュールのインポート-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>必要なモジュールのインポート</a></span></li><li><span><a href="#ノートブックインスタンスの-IAM-ロールに権限を追加" data-toc-modified-id="ノートブックインスタンスの-IAM-ロールに権限を追加-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>ノートブックインスタンスの IAM ロールに権限を追加</a></span></li></ul></li><li><span><a href="#Setup" data-toc-modified-id="Setup-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Amazon-SageMaker-Experiments-の作成" data-toc-modified-id="Amazon-SageMaker-Experiments-の作成-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Amazon SageMaker Experiments の作成</a></span></li><li><span><a href="#Amazon-SageMaker-Model-Package-Group-の作成（Model-Registry-利用準備）" data-toc-modified-id="Amazon-SageMaker-Model-Package-Group-の作成（Model-Registry-利用準備）-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Amazon SageMaker Model Package Group の作成（Model Registry 利用準備）</a></span></li><li><span><a href="#ベースイメージを-Amazon-ECR-に-push" data-toc-modified-id="ベースイメージを-Amazon-ECR-に-push-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>ベースイメージを Amazon ECR に push</a></span></li><li><span><a href="#ML-パイプラインの中の後処理-Lambda-関数の作成" data-toc-modified-id="ML-パイプラインの中の後処理-Lambda-関数の作成-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>ML パイプラインの中の後処理 Lambda 関数の作成</a></span></li><li><span><a href="#評価結果読み出し-Lambda-関数の作成" data-toc-modified-id="評価結果読み出し-Lambda-関数の作成-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>評価結果読み出し Lambda 関数の作成</a></span></li></ul></li><li><span><a href="#AWS-CodeCommit-リポジトリの作成" data-toc-modified-id="AWS-CodeCommit-リポジトリの作成-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>AWS CodeCommit リポジトリの作成</a></span><ul class="toc-item"><li><span><a href="#create_repository()-API-でリポジトリを作成" data-toc-modified-id="create_repository()-API-でリポジトリを作成-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>create_repository() API でリポジトリを作成</a></span></li></ul></li><li><span><a href="#AWS-CodeBuild-プロジェクトの作成" data-toc-modified-id="AWS-CodeBuild-プロジェクトの作成-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>AWS CodeBuild プロジェクトの作成</a></span><ul class="toc-item"><li><span><a href="#Build-Stage-で使用する-IAM-Policy-と-Role-の作成" data-toc-modified-id="Build-Stage-で使用する-IAM-Policy-と-Role-の作成-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Build Stage で使用する IAM Policy と Role の作成</a></span></li><li><span><a href="#create_project()-API-で-Build-Stage-用プロジェクトを作成" data-toc-modified-id="create_project()-API-で-Build-Stage-用プロジェクトを作成-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>create_project() API で Build Stage 用プロジェクトを作成</a></span></li><li><span><a href="#Deploy-Stage-で使用する-IAM-Policy-と-Role-の作成" data-toc-modified-id="Deploy-Stage-で使用する-IAM-Policy-と-Role-の作成-1.5.3"><span class="toc-item-num">1.5.3&nbsp;&nbsp;</span>Deploy Stage で使用する IAM Policy と Role の作成</a></span></li><li><span><a href="#create_project()-API-で-Deploy-Stage-用プロジェクトを作成" data-toc-modified-id="create_project()-API-で-Deploy-Stage-用プロジェクトを作成-1.5.4"><span class="toc-item-num">1.5.4&nbsp;&nbsp;</span>create_project() API で Deploy Stage 用プロジェクトを作成</a></span></li><li><span><a href="#AWS-Step-Functions-用-IAM-Policy-と-Role-の作成" data-toc-modified-id="AWS-Step-Functions-用-IAM-Policy-と-Role-の作成-1.5.5"><span class="toc-item-num">1.5.5&nbsp;&nbsp;</span>AWS Step Functions 用 IAM Policy と Role の作成</a></span></li><li><span><a href="#ML-パイプライン設定ファイルの作成" data-toc-modified-id="ML-パイプライン設定ファイルの作成-1.5.6"><span class="toc-item-num">1.5.6&nbsp;&nbsp;</span>ML パイプライン設定ファイルの作成</a></span></li><li><span><a href="#データ準備用のコンテナイメージ用-Dockerfile-の作成" data-toc-modified-id="データ準備用のコンテナイメージ用-Dockerfile-の作成-1.5.7"><span class="toc-item-num">1.5.7&nbsp;&nbsp;</span>データ準備用のコンテナイメージ用 Dockerfile の作成</a></span></li><li><span><a href="#モデル学習用のコンテナイメージ用-Dockerfile-の作成" data-toc-modified-id="モデル学習用のコンテナイメージ用-Dockerfile-の作成-1.5.8"><span class="toc-item-num">1.5.8&nbsp;&nbsp;</span>モデル学習用のコンテナイメージ用 Dockerfile の作成</a></span></li><li><span><a href="#モデル評価用のコンテナイメージ用-Dockerfile-の作成" data-toc-modified-id="モデル評価用のコンテナイメージ用-Dockerfile-の作成-1.5.9"><span class="toc-item-num">1.5.9&nbsp;&nbsp;</span>モデル評価用のコンテナイメージ用 Dockerfile の作成</a></span></li><li><span><a href="#モデルデプロイ用のコンテナイメージ用-Dockerfile-の作成" data-toc-modified-id="モデルデプロイ用のコンテナイメージ用-Dockerfile-の作成-1.5.10"><span class="toc-item-num">1.5.10&nbsp;&nbsp;</span>モデルデプロイ用のコンテナイメージ用 Dockerfile の作成</a></span></li><li><span><a href="#作成した-CodeBuild-用のファイルを-CodeCommit-に-push" data-toc-modified-id="作成した-CodeBuild-用のファイルを-CodeCommit-に-push-1.5.11"><span class="toc-item-num">1.5.11&nbsp;&nbsp;</span>作成した CodeBuild 用のファイルを CodeCommit に push</a></span></li></ul></li><li><span><a href="#CodePipeline-のパイプライン作成" data-toc-modified-id="CodePipeline-のパイプライン作成-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>CodePipeline のパイプライン作成</a></span><ul class="toc-item"><li><span><a href="#CodePipeline-で使用する-IAM-Policy-と-Role-を作成" data-toc-modified-id="CodePipeline-で使用する-IAM-Policy-と-Role-を作成-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>CodePipeline で使用する IAM Policy と Role を作成</a></span></li><li><span><a href="#create_pipeline()-API-でパイプラインを作成" data-toc-modified-id="create_pipeline()-API-でパイプラインを作成-1.6.2"><span class="toc-item-num">1.6.2&nbsp;&nbsp;</span>create_pipeline() API でパイプラインを作成</a></span></li></ul></li><li><span><a href="#記録された各種データの確認" data-toc-modified-id="記録された各種データの確認-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>記録された各種データの確認</a></span><ul class="toc-item"><li><span><a href="#Amazon-Experiments-の中を確認" data-toc-modified-id="Amazon-Experiments-の中を確認-1.7.1"><span class="toc-item-num">1.7.1&nbsp;&nbsp;</span>Amazon Experiments の中を確認</a></span></li><li><span><a href="#Model-Registry-の中を確認" data-toc-modified-id="Model-Registry-の中を確認-1.7.2"><span class="toc-item-num">1.7.2&nbsp;&nbsp;</span>Model Registry の中を確認</a></span></li></ul></li><li><span><a href="#推論エンドポイントの動作確認" data-toc-modified-id="推論エンドポイントの動作確認-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>推論エンドポイントの動作確認</a></span></li><li><span><a href="#[おまけ]-Model-Registry-のモデルパッケージをデプロイ" data-toc-modified-id="[おまけ]-Model-Registry-のモデルパッケージをデプロイ-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>[おまけ] Model Registry のモデルパッケージをデプロイ</a></span></li><li><span><a href="#リソースの削除" data-toc-modified-id="リソースの削除-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>リソースの削除</a></span><ul class="toc-item"><li><span><a href="#Amazon-SageMaker-推論エンドポイントの削除" data-toc-modified-id="Amazon-SageMaker-推論エンドポイントの削除-1.10.1"><span class="toc-item-num">1.10.1&nbsp;&nbsp;</span>Amazon SageMaker 推論エンドポイントの削除</a></span></li><li><span><a href="#CodeCommit,-CodeBuild,-CodePipeline-の削除" data-toc-modified-id="CodeCommit,-CodeBuild,-CodePipeline-の削除-1.10.2"><span class="toc-item-num">1.10.2&nbsp;&nbsp;</span>CodeCommit, CodeBuild, CodePipeline の削除</a></span></li><li><span><a href="#Amazon-ECR-リポジトリの削除" data-toc-modified-id="Amazon-ECR-リポジトリの削除-1.10.3"><span class="toc-item-num">1.10.3&nbsp;&nbsp;</span>Amazon ECR リポジトリの削除</a></span></li><li><span><a href="#Lambda-関数の削除" data-toc-modified-id="Lambda-関数の削除-1.10.4"><span class="toc-item-num">1.10.4&nbsp;&nbsp;</span>Lambda 関数の削除</a></span></li><li><span><a href="#Amazon-SageMaker-Experiment-の削除" data-toc-modified-id="Amazon-SageMaker-Experiment-の削除-1.10.5"><span class="toc-item-num">1.10.5&nbsp;&nbsp;</span>Amazon SageMaker Experiment の削除</a></span></li><li><span><a href="#Amazon-SageMaker-Model-Registry-の削除" data-toc-modified-id="Amazon-SageMaker-Model-Registry-の削除-1.10.6"><span class="toc-item-num">1.10.6&nbsp;&nbsp;</span>Amazon SageMaker Model Registry の削除</a></span></li><li><span><a href="#AWS-Step-Functions-ワークフローの削除" data-toc-modified-id="AWS-Step-Functions-ワークフローの削除-1.10.7"><span class="toc-item-num">1.10.7&nbsp;&nbsp;</span>AWS Step Functions ワークフローの削除</a></span></li><li><span><a href="#IAM-Role-と-Policy-の削除" data-toc-modified-id="IAM-Role-と-Policy-の削除-1.10.8"><span class="toc-item-num">1.10.8&nbsp;&nbsp;</span>IAM Role と Policy の削除</a></span></li><li><span><a href="#Amazon-S3-バケットの削除" data-toc-modified-id="Amazon-S3-バケットの削除-1.10.9"><span class="toc-item-num">1.10.9&nbsp;&nbsp;</span>Amazon S3 バケットの削除</a></span></li></ul></li><li><span><a href="#[Option]-CI/CD-パイプラインの定期実行" data-toc-modified-id="[Option]-CI/CD-パイプラインの定期実行-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>[Option] CI/CD パイプラインの定期実行</a></span><ul class="toc-item"><li><span><a href="#flow.yml-更新用-Lambda-関数の作成" data-toc-modified-id="flow.yml-更新用-Lambda-関数の作成-1.11.1"><span class="toc-item-num">1.11.1&nbsp;&nbsp;</span>flow.yml 更新用 Lambda 関数の作成</a></span></li><li><span><a href="#定期実行のための-Amazon-EventBridge-Rule-作成" data-toc-modified-id="定期実行のための-Amazon-EventBridge-Rule-作成-1.11.2"><span class="toc-item-num">1.11.2&nbsp;&nbsp;</span>定期実行のための Amazon EventBridge Rule 作成</a></span></li><li><span><a href="#リソースの削除" data-toc-modified-id="リソースの削除-1.11.3"><span class="toc-item-num">1.11.3&nbsp;&nbsp;</span>リソースの削除</a></span></li></ul></li></ul></li></ul></div>

## 環境のセットアップ

このノートブックを実行するのに必要なライブラリをインストールします。

In [ ]:
# Import the latest sagemaker, stepfunctions and boto3 SDKs
import sys

!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install -qU awscli boto3 "sagemaker>=2.0.0"
!{sys.executable} -m pip install -qU "stepfunctions==2.3.0"
!{sys.executable} -m pip install sagemaker-experiments
!{sys.executable} -m pip show sagemaker

このサンプルノートブックは長いので、実行したいセルにアクセスしやすいよう Table of Contents を作成する拡張機能をインストールすると便利です。以下のセルを実行したあと、このノートブックを開いているブラウザのタブをリロードすると Table of Contents の拡張機能が使えるようになります。

In [ ]:
%%sh
pip install jupyter_contrib_nbextensions
jupyter contrib nbextension install --user
jupyter nbextension enable toc2/main

### 必要なモジュールのインポート

同一アカウントで複数の方が同時にこのノートブックを実行する場合は、以下のセルの一番下の行の `user_name` に各自のお名前を設定してください。

In [ ]:
import boto3
from datetime import datetime
from dateutil import tz
import json
import os
import sagemaker
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from stepfunctions.workflow import Workflow
from time import sleep

JST = tz.gettz('Asia/Tokyo')
timestamp = datetime.now(JST).strftime('%Y%m%d-%H%M%S')

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')

codecommit_client = boto3.client('codecommit', region_name=region)
codebuild_client = boto3.client('codebuild', region_name=region)
codepipeline_client = boto3.client('codepipeline', region_name=region)
iam_client = boto3.client('iam', region_name=region)
s3_client = boto3.client('s3', region_name=region)
lambda_client = boto3.client('lambda', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
sagemaker_client = boto3.client('sagemaker', region_name=region)

以下のセルを実行して、各種リソース名を作成します。

In [ ]:
project_name = 'mlops-pipeline'
user_name = 'sample'
bucket_name = project_name + '-' + user_name + '-' + timestamp
code_repository_name = project_name + '-code-' + user_name
codebuild_project_name = project_name + '-codebuild-' + user_name
codebuild_deploy_project_name = project_name + '-codebuild-deploy-' + user_name
codepipeline_name = project_name + '-codepipeline-' + user_name
mlworkflow_name = project_name + '-flow-' + user_name
postprocess_name = project_name + '-post-' + user_name
experiment_name = project_name + '-exp-' + user_name
model_package_group_name = project_name + '-mpg-' + user_name
lambda_read_metrics_function_name = project_name + '-read-metrics-' + user_name
role_name_list = []
policy_arn_list = []
lambda_function_list = []

IAM 関連の関数を作成します。

In [ ]:
def get_policy_arn(policy_name):
    next_token = ''
    while True:
        if next_token == '':
            response = iam_client.list_policies(Scope='Local')
        else:
            response = iam_client.list_policies(Scope='Local', Marker=next_token)
        for content in response['Policies']:
            if policy_name == content['PolicyName']:
                return content['Arn']
        if 'Marker' in response:
            next_token = response['Marker']
        else:
            break

    return ''


def get_role_arn(role_name):
    next_token = ''
    while True:
        if next_token == '':
            response = iam_client.list_roles()
        else:
            response = iam_client.list_roles(Marker=next_token)
        for content in response['Roles']:
            if role_name == content['RoleName']:
                return content['Arn']
        if 'Marker' in response:
            next_token = response['Marker']
        else:
            break

    return ''


def create_role(role_name, assume_role_policy):
    try:
        response = iam_client.create_role(
            Path = '/service-role/',
            RoleName = role_name,
            AssumeRolePolicyDocument = json.dumps(assume_role_policy),
            MaxSessionDuration=3600*12 # 12 hours
        )
        role_arn = response['Role']['Arn']
    except Exception as ex:
        if "EntityAlreadyExists" in str(ex):
            detach_role_policies(role_name)
            response = iam_client.delete_role(
                RoleName = role_name,
            )
            response = iam_client.create_role(
                Path = '/service-role/',
                RoleName = role_name,
                AssumeRolePolicyDocument = json.dumps(assume_role_policy),
                MaxSessionDuration=3600*12 # 12 hours
            )
            role_arn = response['Role']['Arn']
    sleep(10)
    return role_arn


def create_policy(policy_name, inline_policy):
    try:
        response = iam_client.create_policy(
            PolicyName=policy_name,
            PolicyDocument=json.dumps(inline_policy),
        )
        policy_arn = response['Policy']['Arn']
    except Exception as ex:
        if "EntityAlreadyExists" in str(ex):
            response = iam_client.delete_policy(
                PolicyArn=get_policy_arn(policy_name)
            )
            response = iam_client.create_policy(
                PolicyName=policy_name,
                PolicyDocument=json.dumps(inline_policy),
            )
            policy_arn = response['Policy']['Arn']
    policy_arn_list.append(policy_arn)
    sleep(10)
    return policy_arn


def create_policy_role(policy_name, inline_policy, role_name, assume_role_policy, policy_list=[]):

    role_arn = create_role(role_name, assume_role_policy)
    policy_arn = create_policy(policy_name, inline_policy)

    sleep(5)
    response = iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn=policy_arn
    )

    for p in policy_list:
        response = iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/' + p
        )

    role_name_list.append(role_name)
    policy_arn_list.append(policy_arn)
    sleep(10)

In [ ]:
def get_model_packages(model_package_group_name):
    next_token = ''
    model_package_list = []
    while True:
        if next_token == '':
            response = sagemaker_client.list_model_packages(
                            ModelPackageGroupName=model_package_group_name,
                        )
        else:
            response = sagemaker_client.list_model_packages(
                            ModelPackageGroupName=model_package_group_name,
                            NextToken=next_token,
                        )
        for content in response['ModelPackageSummaryList']:
            model_package_list.append(content)
        if 'NextToken' in response:
            next_token = response['Marker']
        else:
            break

    return model_package_list

### ノートブックインスタンスの IAM ロールに権限を追加

以下の手順を実行して、ノートブックインスタンスに紐づけられた IAM ロールに、AWS Step Functions のワークフローを作成して実行するための権限と Amazon ECR にイメージを push するための権限を追加してください。


1. [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/) を開く
1. **ノートブックインスタンス** を開いて現在使用しているノートブックインスタンスを選択する
1. **アクセス許可と暗号化** の部分に表示されている IAM ロールへのリンクをクリックする
1. IAM ロールの ARN は後で使用するのでメモ帳などにコピーしておく
1. **ポリシーをアタッチします** をクリックして `IAMFullAccess` を検索する
1. `IAMFullAccess` の横のチェックボックスをオンにする
1. **ポリシーのアタッチ** をクリックする


もしこのノートブックを SageMaker のノートブックインスタンス以外で実行している場合、その環境で AWS CLI 設定を行ってください。詳細は [Configuring the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) をご参照ください。

In [ ]:
policy_list = ['AWSCodePipeline_FullAccess', 'AmazonEC2ContainerRegistryFullAccess',
               'AWSLambda_FullAccess', 'AWSCodeCommitFullAccess',
              'AWSCodeBuildAdminAccess', 'AWSStepFunctionsFullAccess']

inline_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "s3:PutEncryptionConfiguration",
            "Resource": "arn:aws:s3:::" + bucket_name
        }
    ]
}

sagemaker_policy_name = project_name + '-' + user_name + '-policy'
sagemaker_policy_arn = get_policy_arn(sagemaker_policy_name)

if len(sagemaker_policy_arn) > 0:
    response = iam_client.detach_role_policy(
        RoleName=role.split('/')[2],
        PolicyArn=sagemaker_policy_arn
    )
sagemaker_policy_arn = create_policy(sagemaker_policy_name, inline_policy)
sleep(5)

response = iam_client.attach_role_policy(
        RoleName=role.split('/')[2],
        PolicyArn=sagemaker_policy_arn
    )

for l in policy_list:
    response = iam_client.attach_role_policy(
        RoleName=role.split('/')[2],
        PolicyArn='arn:aws:iam::aws:policy/' + l
    )
sleep(10)

以下のセルを実行して、このサンプルでファイルの保存に使用するバケットを新規作成します。

In [ ]:
if region == 'us-east-1':
    response = s3_client.create_bucket(Bucket=bucket_name)
else:
    location = {'LocationConstraint': region}
    response = s3_client.create_bucket(Bucket=bucket_name,
                                       CreateBucketConfiguration=location)
sleep(10)
response = s3_client.put_bucket_encryption(
    Bucket=bucket_name,
    ServerSideEncryptionConfiguration={
        'Rules': [
            {
                'ApplyServerSideEncryptionByDefault': {
                    'SSEAlgorithm': 'AES256',
                },
            },
        ]
    },
)

作成したバケットの暗号化を有効化します。

In [ ]:
response = s3_client.put_bucket_encryption(
    Bucket=bucket_name,
    ServerSideEncryptionConfiguration={
        'Rules': [
            {
                'ApplyServerSideEncryptionByDefault': {
                    'SSEAlgorithm': 'AES256',
                },
            },
        ]
    },
)

## Setup
### Amazon SageMaker Experiments の作成

In [ ]:
# create the experiment if it doesn't exist
try:
    experiment_evaluate = Experiment.load(experiment_name=experiment_name)
except Exception as ex:
    if "ResourceNotFound" in str(ex):
        experiment_evaluate = Experiment.create(
            experiment_name=experiment_name, 
            description="model evaluation", 
            sagemaker_boto_client=boto3.client('sagemaker'))
    else:
        print(ex)

print(experiment_evaluate.experiment_name)

### Amazon SageMaker Model Package Group の作成（Model Registry 利用準備）

In [ ]:
model_package_group_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": "This is MLOps demo",
}

try:
    create_model_pacakge_group_response = sagemaker_client.create_model_package_group(
        **model_package_group_input_dict
    )
    model_package_group_arn = create_model_pacakge_group_response["ModelPackageGroupArn"]
except Exception as e:
    model_package_group_arn = f'arn:aws:sagemaker:{region}:{account_id}:model-package-group/{model_package_group_name}'
print(f"ModelPackageGroup Arn : {model_package_group_arn}")

### ベースイメージを Amazon ECR に push

試行錯誤の中で何度もコンテナイメージのビルドを繰り返していると、Docker Hub からベースイメージを pull できなくなることがあります。そこで、はじめにベースイメージをビルドして Amazon ECR に push しておき、そちらを今後ベースイメージとして使用します。

In [ ]:
def build_and_push_image(repo_name, docker_path, extra_accounts=[], tag = ':latest'):
    uri_suffix = 'amazonaws.com'
    repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, repo_name + tag)

    !docker build -t $repo_name $docker_path
    for a in extra_accounts:
        !aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {a}.dkr.ecr.{region}.amazonaws.com
    !aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
    !aws ecr create-repository --repository-name $repo_name
    !docker tag {repo_name + tag} $repository_uri
    !docker push $repository_uri
    return repository_uri

In [ ]:
base_repository_uri = build_and_push_image(project_name + '-base-' + user_name, '.')

### ML パイプラインの中の後処理 Lambda 関数の作成

モデルの評価ジョブでの評価結果をもとに、Lambda 関数で後処理を行います。後処理としては例えば、Slack にモデルの評価結果をポストしたり、別の Workflow を起動したりなどが考えられます。このノートブックでは、評価結果に応じて表示するテキストの内容を変えています。

実行環境やソースコードが入ったコンテナイメージを Lambda にデプロイします。このノートブックでは、Experiment で管理されている情報を取得するために `sagemaker.analytics.ExperimentAnalytics` を使用するため、必要なライブラリが入ったコンテナを使って Lambda 関数を実行します。

In [ ]:
dockerfile_name = 'docker/postprocess/Dockerfile'

dockerfile_code = f"""FROM {base_repository_uri}

# Include global arg in this stage of the build
ARG FUNCTION_DIR="/function"

COPY requirements.txt .
RUN pip3 install --upgrade pip
RUN pip3 install -qU -r requirements.txt

# Copy function code
RUN mkdir -p $FUNCTION_DIR
COPY app/ $FUNCTION_DIR/

# Set working directory to function root directory
WORKDIR $FUNCTION_DIR


ENTRYPOINT [ "/usr/local/bin/python", "-m", "awslambdaric" ]
CMD [ "app.handler" ]
"""

with open(dockerfile_name, 'w') as f:
    f.write(dockerfile_code)

In [ ]:
lambda_repository_uri = build_and_push_image(project_name + '-lambda-' + user_name, 'docker/postprocess')

In [ ]:
import boto3
import json


def detach_role_policies(role_name):
    response = iam_client.list_attached_role_policies(
        RoleName=role_name,
    )
    policies = response['AttachedPolicies']

    for p in policies:
        response = iam_client.detach_role_policy(
            RoleName=role_name,
            PolicyArn=p['PolicyArn']
        )

        
def function_exists(function_name):
    next_token = ''
    while True:
        if next_token == '':
            response = lambda_client.list_functions()
        else:
            response = lambda_client.list_functions(Marker=next_token)
        for content in response['Functions']:
            if function_name == content['FunctionName']:
                return True
        if 'NextMarker' in response:
            next_token = response['NextMarker']
        else:
            break

    return False


def create_container_lambda_function(function_name, image_uri, role_arn):

    if function_exists(function_name):
        response = lambda_client.delete_function(
            FunctionName=function_name,
        )
        sleep(10)

    response = lambda_client.create_function(
        FunctionName=function_name,
        Role=role_arn,
        Code={
            'ImageUri':image_uri
        },
        Timeout=60*5, # 5 minutes
        MemorySize=128, # 128 MB
        Publish=True,
        PackageType='Image',
    )
    lambda_function_list.append(function_name)

In [ ]:
lambda_policy_name = postprocess_name + '-policy'
lambda_role_name = postprocess_name + '-role'

inline_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "sagemaker:Search",
            "Resource": "*"
        },
        {
            'Effect': 'Allow',
            'Action': 'logs:CreateLogGroup',
            'Resource': f'arn:aws:logs:{region}:{account_id}:*'
        },
        {
            'Effect': 'Allow',
            'Action': [
                'logs:CreateLogStream',
                'logs:PutLogEvents'
            ],
            'Resource': [
                f'arn:aws:logs:{region}:{account_id}:log-group:/aws/lambda/{postprocess_name}:*'
            ]
        }
    ]
}

assume_role_policy = {
  "Version": "2012-10-17",
  "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"lambda.amazonaws.com"},"Action": "sts:AssumeRole"}]
}

create_policy_role(lambda_policy_name, inline_policy,
                   lambda_role_name, assume_role_policy)
lambda_role_arn = iam_client.get_role(RoleName=lambda_role_name)['Role']['Arn']
sleep(10) # wait until IAM is created
create_container_lambda_function(postprocess_name, lambda_repository_uri, lambda_role_arn)

### 評価結果読み出し Lambda 関数の作成

In [ ]:
def create_lambda_function(function_name, file_name, role_arn, py_version='python3.9'):

    with open(file_name+'.zip', 'rb') as f:
        zip_data = f.read()
        
    if function_exists(function_name):
        response = lambda_client.update_function_code(
            FunctionName=function_name,
            ZipFile=zip_data,
            Publish=True,
        )
    else:
        response = lambda_client.create_function(
            FunctionName=function_name,
            Role=role_arn,
            Handler=file_name+'.lambda_handler',
            Runtime=py_version,
            Code={
                'ZipFile':zip_data
            },
            Timeout=60*5, # 5 minutes
            MemorySize=128, # 128 MB
            Publish=True,
            PackageType='Zip',
        )
    lambda_function_list.append(function_name)
    return response['FunctionArn']

In [ ]:
lambda_read_metrics_policy_name = lambda_read_metrics_function_name + '-policy'
lambda_read_metrics_role_name = lambda_read_metrics_function_name + '-role'

inline_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "codepipeline:PutJobSuccessResult",
            "Resource": "*"
        },
        {
            'Effect': 'Allow',
            'Action': 'logs:CreateLogGroup',
            'Resource': f'arn:aws:logs:{region}:{account_id}:*'
        },
        {
            'Effect': 'Allow',
            'Action': [
                'logs:CreateLogStream',
                'logs:PutLogEvents'
            ],
            'Resource': [
                f'arn:aws:logs:{region}:{account_id}:log-group:/aws/lambda/{lambda_read_metrics_function_name}:*'
            ]
        }
    ]
}

assume_role_policy = {
  "Version": "2012-10-17",
  "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"lambda.amazonaws.com"},"Action": "sts:AssumeRole"}]
}

policy_list=['AmazonS3ReadOnlyAccess']

create_policy_role(lambda_read_metrics_policy_name, inline_policy,
                   lambda_read_metrics_role_name, assume_role_policy, policy_list=policy_list)
lambda_read_metrics_role_arn = iam_client.get_role(RoleName=lambda_read_metrics_role_name)['Role']['Arn']
sleep(10) # wait until IAM is created

import zipfile
file_name = 'read_metrics'
with zipfile.ZipFile(file_name+'.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
    new_zip.write(file_name+'.py')

create_lambda_function(lambda_read_metrics_function_name, file_name, lambda_read_metrics_role_arn)

## AWS CodeCommit リポジトリの作成

### create_repository() API でリポジトリを作成

API の詳細は[こちらのドキュメント](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/codecommit.html#CodeCommit.Client.create_repository) をご参照ください。

In [ ]:
try:
    response = codecommit_client.create_repository(
        repositoryName=code_repository_name,
        repositoryDescription='sample repository',
        tags={
            'owner': user_name
        }
    )
except Exception as e:
    print(e)

学習パイプラインを作成するためのコード一式を、作成したリポジトリに push します。

In [ ]:
%%sh -s $code_repository_name $region $user_name
echo $1
rm -rf $1
git clone https://git-codecommit.$2.amazonaws.com/v1/repos/$1
cd $1
git config --global user.name $3
git config --global user.email you@example.com
git checkout -b main
cp -r ../git-repo-codes/* ./
# touch readme.md
git add .
git commit -m "first commit"
git push --set-upstream origin main

## AWS CodeBuild プロジェクトの作成

### Build Stage で使用する IAM Policy と Role の作成

以下のセルを実行して、CodeBuild が使用する IAM Policy と Role を作成します。

In [ ]:
inline_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "iam:PassRole"
            ],
            "Effect": "Allow",
            "Resource": "*"
        }
    ]
}

assume_role_policy = {
          "Version": "2012-10-17",
          "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"codebuild.amazonaws.com"},"Action": "sts:AssumeRole"}]
        }

codebuild_policy_name = codebuild_project_name + '-policy'
codebuild_role_name = codebuild_project_name + '-role'

create_policy_role(codebuild_policy_name, inline_policy,
                   codebuild_role_name, assume_role_policy,
                   policy_list=['AmazonEC2ContainerRegistryFullAccess',
                       'AmazonS3FullAccess',
                       'AWSCodeBuildDeveloperAccess',
                       'AWSStepFunctionsFullAccess',
                       'CloudWatchLogsFullAccess',
                       'AWSLambda_FullAccess'])

### create_project() API で Build Stage 用プロジェクトを作成

API の詳細は [こちらのドキュメント](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/codebuild.html#CodeBuild.Client.create_project) をご参照ください。

In [ ]:
codebuild_role_arn = get_role_arn(codebuild_role_name)

response = codebuild_client.create_project(
    name=codebuild_project_name,
    description='sample project',
    source={
        'type': 'CODEPIPELINE',
        'insecureSsl': False,
    },
    artifacts={
        'type': 'CODEPIPELINE',
        'encryptionDisabled': False,
    },
    cache={
        'type': 'LOCAL',
        'modes': [
            'LOCAL_DOCKER_LAYER_CACHE',
        ]
    },
    environment={
        'type': 'LINUX_CONTAINER',
        'image': 'aws/codebuild/standard:5.0',
        'computeType': 'BUILD_GENERAL1_MEDIUM',
        'privilegedMode': True,
        'imagePullCredentialsType': 'CODEBUILD'
    },
    serviceRole=codebuild_role_arn,
    timeoutInMinutes=120,
    queuedTimeoutInMinutes=120,
#     encryptionKey='string',
    tags=[
        {
            'key': 'owner',
            'value': user_name
        },
    ],
    badgeEnabled=False,
    logsConfig={
        'cloudWatchLogs': {
            'status': 'ENABLED',
        },
        's3Logs': {
            'status': 'ENABLED',
            'location': os.path.join(bucket_name, 'build-log'),
            'encryptionDisabled': False,
            'bucketOwnerAccess': 'READ_ONLY'
        }
    },
)

### Deploy Stage で使用する IAM Policy と Role の作成

In [ ]:
inline_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "iam:PassRole"
            ],
            "Effect": "Allow",
            "Resource": "*"
        }
    ]
}

assume_role_policy = {
          "Version": "2012-10-17",
          "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"codebuild.amazonaws.com"},"Action": "sts:AssumeRole"}]
        }

codebuild_deploy_policy_name = codebuild_deploy_project_name + '-policy'
codebuild_deploy_role_name = codebuild_deploy_project_name + '-role'

create_policy_role(codebuild_deploy_policy_name, inline_policy,
                   codebuild_deploy_role_name, assume_role_policy,
                   policy_list=['AmazonEC2ContainerRegistryFullAccess',
                       'AmazonS3FullAccess',
                       'AWSCodeBuildDeveloperAccess',
                       'AmazonSageMakerFullAccess',
                       'CloudWatchLogsFullAccess',
                       'AWSCodePipelineReadOnlyAccess'])

### create_project() API で Deploy Stage 用プロジェクトを作成

In [ ]:
codebuild_deploy_role_arn = get_role_arn(codebuild_deploy_role_name)

response = codebuild_client.create_project(
    name=codebuild_deploy_project_name,
    description='sample deploy project',
    source={
        'type': 'CODEPIPELINE',
        'buildspec': 'buildspec-deploy.yml',
        'insecureSsl': False,
    },
    artifacts={
        'type': 'CODEPIPELINE',
        'encryptionDisabled': False,
    },
    cache={
        'type': 'LOCAL',
#         'location': 'string',
        'modes': [
            'LOCAL_DOCKER_LAYER_CACHE',
        ]
    },
    environment={
        'type': 'LINUX_CONTAINER',
        'image': 'aws/codebuild/standard:5.0',
        'computeType': 'BUILD_GENERAL1_MEDIUM',
        'privilegedMode': True,
        'imagePullCredentialsType': 'CODEBUILD'
    },
    serviceRole=codebuild_deploy_role_arn,
    timeoutInMinutes=120,
    queuedTimeoutInMinutes=120,
#     encryptionKey='string',
    tags=[
        {
            'key': 'owner',
            'value': user_name
        },
    ],
    badgeEnabled=False,
    logsConfig={
        'cloudWatchLogs': {
            'status': 'ENABLED',
#             'groupName': 'string',
#             'streamName': 'string'
        },
        's3Logs': {
            'status': 'ENABLED',
            'location': os.path.join(bucket_name, 'build-log'),
            'encryptionDisabled': False,
            'bucketOwnerAccess': 'READ_ONLY'
        }
    },
)

### AWS Step Functions 用 IAM Policy と Role の作成

In [ ]:
import json

step_functions_policy_name = project_name + '-stepfunctions-' + user_name + '-policy'
step_functions_role_name = project_name + '-stepfunctions-' + user_name + '-role'
inline_policy ={
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:DescribeRule",
                "events:PutRule"
            ],
            "Resource": [
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTrainingJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTransformJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTuningJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForECSTaskRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForBatchJobsRule"
            ]
        },
        {
            "Sid": "VisualEditor1",
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": role,
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "sagemaker.amazonaws.com"
                }
            }
        },
        {
            "Sid": "VisualEditor2",
            "Effect": "Allow",
            "Action": [
                "ecs:DescribeTasks",
                "ecs:RunTask",
                "ecs:StopTask",
                "lambda:InvokeFunction",
                "sagemaker:CreateEndpoint",
                "sagemaker:CreateEndpointConfig",
                "sagemaker:CreateHyperParameterTuningJob",
                "sagemaker:CreateModel",
                "sagemaker:CreateProcessingJob",
                "sagemaker:CreateTrainingJob",
                "sagemaker:CreateTransformJob",
                "sagemaker:DeleteEndpoint",
                "sagemaker:DeleteEndpointConfig",
                "sagemaker:DescribeHyperParameterTuningJob",
                "sagemaker:DescribeProcessingJob",
                "sagemaker:DescribeTrainingJob",
                "sagemaker:DescribeTransformJob",
                "sagemaker:ListProcessingJobs",
                "sagemaker:ListTags",
                "sagemaker:StopHyperParameterTuningJob",
                "sagemaker:StopProcessingJob",
                "sagemaker:StopTrainingJob",
                "sagemaker:StopTransformJob",
                "sagemaker:UpdateEndpoint",
                "sns:Publish",
                "sqs:SendMessage"
            ],
            "Resource": "*"
        }
    ]
}

assume_role_policy = {
      "Version": "2012-10-17",
      "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"states.amazonaws.com"},"Action": "sts:AssumeRole"}]
    }

create_policy_role(step_functions_policy_name, inline_policy,
                   step_functions_role_name, assume_role_policy,
                   ['CloudWatchEventsFullAccess'])

### ML パイプライン設定ファイルの作成
ML パイプラインの設定値が記載された flow.yml を作成します。この設定ファイルには、学習データが保存されている Amazon S3 パスや機械学習アルゴリズムのハイパーパラメタなどの、機械学習モデルを学習。評価するために必要な情報が記載されています。パラメタを変えて CI/CD パイプラインを実行したい場合はこの flow.yml ファイルを更新して CodeCommit に push することで CodePipeline のパイプラインが開始します。

In [ ]:
step_functions_role_arn = get_role_arn(step_functions_role_name)
filepath = os.path.join(code_repository_name, 'flow.yml')

flow_yml={f"""config:
  user-name: {user_name}
  region: {region}
  codepipeline-name: {codepipeline_name}
  job-name-prefix: {mlworkflow_name}
  image-name-prefix: {mlworkflow_name}
  sfn-workflow-name: {mlworkflow_name}
  sagemaker-experiment-name: {experiment_name}
  sfn-role-arn: {step_functions_role_arn}
  sagemaker-role: {role}
  framework-version: 1.9.1
  model-package-group-arn: {model_package_group_arn}
preprocess:
  image-repo-name: {project_name}-preprocess-{user_name}
  input-data-path: s3://fast-ai-imageclas/mnist_png.tgz
  output-data-path: s3://{bucket_name}/data/PennFudanPed_Augmented
train:
  image-repo-name: {project_name}-train-{user_name}
  output-path: s3://{bucket_name}/train
  hyperparameters:
    batch-size: 4
    epoch: 2
evaluate:
  image-repo-name: {project_name}-evaluate-{user_name}
  data-path: s3://{bucket_name}/data
  result-path: s3://{bucket_name}/evaluate
postprocess:
  lambda-func-name: {postprocess_name}
inference:
  image-repo-name: {project_name}-inference-{user_name}
"""
}

with open(filepath, 'w') as f:
    f.write('\n'.join(list(flow_yml)))

### データ準備用のコンテナイメージ用 Dockerfile の作成
データ準備用のコンテナイメージを作るための Dockerfile を作成します。

In [ ]:
filepath = os.path.join(code_repository_name, 'ml-pipeline/data-preparation/Dockerfile')

prep_dockerfile={f"""FROM {base_repository_uri}
    
ENV AWS_DEFAULT_REGION {region}

COPY requirements.txt .
COPY preprocessing.py /opt/ml/processing/code/
RUN pip3 install --upgrade pip
RUN pip3 install -qU -r requirements.txt

ENTRYPOINT ["python3", "/opt/ml/processing/code/preprocessing.py"]
"""
}

with open(filepath, 'w') as f:
    f.write('\n'.join(list(prep_dockerfile)))

### モデル学習用のコンテナイメージ用 Dockerfile の作成
次に、モデル学習用のコンテナイメージを作るための Dockerfile を作成します。

In [ ]:
filepath = os.path.join(code_repository_name, 'ml-pipeline/train/Dockerfile')

train_dockerfile={f"""FROM 763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training:1.9.1-gpu-py38-cu111-ubuntu20.04
 
COPY requirements.txt ./
RUN pip uninstall torch torchvision -y && pip install --no-cache-dir -U -r requirements.txt

COPY *.py /opt/ml/code/
ENV SAGEMAKER_PROGRAM train.py
"""
}

with open(filepath, 'w') as f:
    f.write('\n'.join(list(train_dockerfile)))

### モデル評価用のコンテナイメージ用 Dockerfile の作成
続いて、モデル評価用のコンテナイメージを作るための Dockerfile を作成します。

In [ ]:
filepath = os.path.join(code_repository_name, 'ml-pipeline/model-evaluation/Dockerfile')

eval_dockerfile={f"""FROM {base_repository_uri}
    
ENV AWS_DEFAULT_REGION {region}

COPY requirements.txt .
COPY evaluation.py /opt/ml/processing/input/code/
RUN pip3 install --upgrade pip
RUN pip3 install -qU -r requirements.txt

ENTRYPOINT ["python3", "/opt/ml/processing/input/code/evaluation.py"]
"""
}

with open(filepath, 'w') as f:
    f.write('\n'.join(list(eval_dockerfile)))

### モデルデプロイ用のコンテナイメージ用 Dockerfile の作成
最後に、推論エンドポイント用コンテナイメージを作るための Dockerfile を作成します。

In [ ]:
filepath = os.path.join(code_repository_name, 'ml-pipeline/inference/Dockerfile')

inference_dockerfile={f"""FROM 763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-inference:1.9.1-cpu-py38-ubuntu20.04

COPY requirements.txt .
RUN pip3 install qU -r requirements.txt
"""
}

with open(filepath, 'w') as f:
    f.write('\n'.join(list(inference_dockerfile)))

### 作成した CodeBuild 用のファイルを CodeCommit に push

以下のセルを実行して、作成したファイルを CodeCommit に push します。

In [ ]:
%%sh -s $code_repository_name
echo $1
cd $1
git add .
git commit -m "add Dockerfile"
git push

## CodePipeline のパイプライン作成
### CodePipeline で使用する IAM Policy と Role を作成

In [ ]:
codepipeline_policy_name = codepipeline_name + '-policy'
codepipeline_role_name = codepipeline_name + '-role'

inline_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Condition": {
                "StringEqualsIfExists": {
                    "iam:PassedToService": [
                        "cloudformation.amazonaws.com",
                        "elasticbeanstalk.amazonaws.com",
                        "ec2.amazonaws.com",
                        "ecs-tasks.amazonaws.com"
                    ]
                }
            }
        },
        {
            "Action": [
                "codecommit:CancelUploadArchive",
                "codecommit:GetBranch",
                "codecommit:GetCommit",
                "codecommit:GetRepository",
                "codecommit:GetUploadArchiveStatus",
                "codecommit:UploadArchive"
            ],
            "Resource": "*",
            "Effect": "Allow"
        },
        {
            "Action": [
                "codedeploy:CreateDeployment",
                "codedeploy:GetApplication",
                "codedeploy:GetApplicationRevision",
                "codedeploy:GetDeployment",
                "codedeploy:GetDeploymentConfig",
                "codedeploy:RegisterApplicationRevision"
            ],
            "Resource": "*",
            "Effect": "Allow"
        },
        {
            "Action": [
                "codestar-connections:UseConnection"
            ],
            "Resource": "*",
            "Effect": "Allow"
        },
        {
            "Action": [
                "elasticbeanstalk:*",
                "ec2:*",
                "elasticloadbalancing:*",
                "autoscaling:*",
                "cloudwatch:*",
                "s3:*",
                "sns:*",
                "cloudformation:*",
                "rds:*",
                "sqs:*",
                "ecs:*"
            ],
            "Resource": "*",
            "Effect": "Allow"
        },
        {
            "Action": [
                "lambda:InvokeFunction",
                "lambda:ListFunctions"
            ],
            "Resource": "*",
            "Effect": "Allow"
        },
        {
            "Action": [
                "opsworks:CreateDeployment",
                "opsworks:DescribeApps",
                "opsworks:DescribeCommands",
                "opsworks:DescribeDeployments",
                "opsworks:DescribeInstances",
                "opsworks:DescribeStacks",
                "opsworks:UpdateApp",
                "opsworks:UpdateStack"
            ],
            "Resource": "*",
            "Effect": "Allow"
        },
        {
            "Action": [
                "cloudformation:CreateStack",
                "cloudformation:DeleteStack",
                "cloudformation:DescribeStacks",
                "cloudformation:UpdateStack",
                "cloudformation:CreateChangeSet",
                "cloudformation:DeleteChangeSet",
                "cloudformation:DescribeChangeSet",
                "cloudformation:ExecuteChangeSet",
                "cloudformation:SetStackPolicy",
                "cloudformation:ValidateTemplate"
            ],
            "Resource": "*",
            "Effect": "Allow"
        },
        {
            "Action": [
                "codebuild:BatchGetBuilds",
                "codebuild:StartBuild",
                "codebuild:BatchGetBuildBatches",
                "codebuild:StartBuildBatch"
            ],
            "Resource": "*",
            "Effect": "Allow"
        },
        {
            "Effect": "Allow",
            "Action": [
                "devicefarm:ListProjects",
                "devicefarm:ListDevicePools",
                "devicefarm:GetRun",
                "devicefarm:GetUpload",
                "devicefarm:CreateUpload",
                "devicefarm:ScheduleRun"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "servicecatalog:ListProvisioningArtifacts",
                "servicecatalog:CreateProvisioningArtifact",
                "servicecatalog:DescribeProvisioningArtifact",
                "servicecatalog:DeleteProvisioningArtifact",
                "servicecatalog:UpdateProduct"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "cloudformation:ValidateTemplate"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "ecr:DescribeImages"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "states:DescribeExecution",
                "states:DescribeStateMachine",
                "states:StartExecution"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "appconfig:StartDeployment",
                "appconfig:StopDeployment",
                "appconfig:GetDeployment"
            ],
            "Resource": "*"
        },
        {
            "Action": "sts:AssumeRole",
            "Resource": "*",
            "Effect": "Allow"
        }
    ]
}

assume_role_policy = {
      "Version": "2012-10-17",
      "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"codepipeline.amazonaws.com"},"Action": "sts:AssumeRole"}]
    }

create_policy_role(codepipeline_policy_name, inline_policy,
                   codepipeline_role_name, assume_role_policy)

### create_pipeline() API でパイプラインを作成

API の詳細は [こちらのドキュメント](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/codepipeline.html#CodePipeline.Client.create_pipeline) をご参照ください。

In [ ]:
def get_pipeline_list():
    file_list = []
    next_token = ''
    while True:
        if next_token == '':
            response = codepipeline_client.list_pipelines()
        else:
            response = codepipeline_client.list_pipelines(nextToken=next_token)
        for content in response['pipelines']:
            key = content['name']
            file_list.append(key)
        if 'nextToken' in response:
            next_token = response['nextToken']
        else:
            break

    return file_list


In [ ]:
step_functions_workflow_arn = f'arn:aws:states:{region}:{account_id}:stateMachine:' + mlworkflow_name
codepipeline_role_arn = get_role_arn(codepipeline_role_name)

pipeline={
    'name': codepipeline_name,
    'roleArn': codepipeline_role_arn,
    'artifactStore': {
        'type': 'S3',
        'location': bucket_name,
    },
    'stages': [
        {
            'name': 'Source',
            'actions': [
                {
                    'name': 'Code',
                    'actionTypeId': {
                        'category': 'Source',
                        'owner': 'AWS',
                        'provider': 'CodeCommit',
                        'version': '1'
                    },
                    'runOrder': 1,
                    'configuration': {
                        'BranchName': "main", 
                        'PollForSourceChanges': "true",
                        'RepositoryName': code_repository_name
                    },
                    'outputArtifacts': [
                        {
                            'name': 'Code'
                        },
                    ],
                },
            ]
        },
        {
            'name': 'Build',
            'actions': [
                {
                    'name': 'CreateMLWorkflow',
                    'actionTypeId': {
                        'category': 'Build',
                        'owner': 'AWS',
                        'provider': 'CodeBuild',
                        'version': '1'
                    },
                    'runOrder': 2,
                    # CodeBuild の configuration 情報はこちら
                    # https://docs.aws.amazon.com/codepipeline/latest/userguide/action-reference-CodeBuild.html
                    'configuration': {
                        "BatchEnabled": "false",
                        "CombineArtifacts": "false",
                        "ProjectName": codebuild_project_name,
                        "PrimarySource": "Code",
                        "EnvironmentVariables": 
                            "[{\"name\":\"EXEC_ID\","
                                "\"value\":\"#{codepipeline.PipelineExecutionId}\","
                                "\"type\":\"PLAINTEXT\"}]"
                    },
                    'inputArtifacts': [
                        {
                            'name': 'Code'
                        },
                    ],
                    'namespace': 'CreateMLWorkflow'
                },
                {
                    'name': 'RunMLWorkflow',
                    'actionTypeId': {
                        'category': 'Invoke',
                        'owner': 'AWS',
                        'provider': 'StepFunctions',
                        'version': '1'
                    },
                    'runOrder': 3,
                    # Step Functions の configuration 情報はこちら
                    # https://docs.aws.amazon.com/ja_jp/codepipeline/latest/userguide/action-reference-StepFunctions.html
                    'configuration': {
                        "StateMachineArn": step_functions_workflow_arn,
                        "InputType": "Literal",
                        "Input": 
                        "{\"PreprocessingJobName\":\"#{CreateMLWorkflow.PREP_JOB_NAME}\","
                        "\"TrainingJobName\":\"#{CreateMLWorkflow.TRAIN_JOB_NAME}\","
                        " \"EvaluationJobName\":\"#{CreateMLWorkflow.EVAL_JOB_NAME}\"}",
                    },
                },
                {
                    'name': 'ReadMetrics',
                    'actionTypeId': {
                        'category': 'Invoke',
                        'owner': 'AWS',
                        'provider': 'Lambda',
                        'version': '1'
                    },
                    'runOrder': 4,
                    # Lambda の configuration 情報はこちら
                    # https://docs.aws.amazon.com/ja_jp/codepipeline/latest/userguide/action-reference-Lambda.html
                    'configuration': {
                        "FunctionName": lambda_read_metrics_function_name,
                        "UserParameters":"{\"s3path\":\"#{CreateMLWorkflow.EVAL_RESULT_PATH}\"}",
                    },
                    'namespace': 'ReadMetrics'
                },
            ]
        },
        {
            'name': 'Approval',
            'actions': [
                {
                    'name': 'ManualApproval',
                    'actionTypeId': {
                        'category': 'Approval',
                        'owner': 'AWS',
                        'provider': 'Manual',
                        'version': '1'
                    },
                    'runOrder': 5,
                    # Manual Approval の configuration 例
                    # https://docs.aws.amazon.com/codepipeline/latest/userguide/reference-pipeline-structure.html#:~:text=The%20following%20example%20shows%20a%20valid%20configuration%20for%20a%20manual%20approval%3A
                    'configuration': {
                        "CustomData": "This Model is good for deploy? "
                                      "average_loss: #{ReadMetrics.average_loss}, "
                                      "accuracy: #{ReadMetrics.accuracy}",
                        "ExternalEntityLink": "https://aws.amazon.com/jp/",
#                         "NotificationArn": "arn:aws:sns:us-west-2:12345EXAMPLE:Notification"
                    },
                },
            ]
        },
        {
            'name': 'Deploy',
            'actions': [
                {
                    'name': 'DeployModel',
                    'actionTypeId': {
                        'category': 'Build',
                        'owner': 'AWS',
                        'provider': 'CodeBuild',
                        'version': '1'
                    },
                    'runOrder': 6,
                    # CodeBuild の configuration 情報はこちら
                    # https://docs.aws.amazon.com/codepipeline/latest/userguide/action-reference-CodeBuild.html
                    'configuration': {
                        "BatchEnabled": "false",
                        "CombineArtifacts": "false",
                        "ProjectName": codebuild_deploy_project_name,
                        "PrimarySource": "Code",
                        "EnvironmentVariables": 
                            "[{\"name\":\"EXEC_ID\","
                                "\"value\":\"#{codepipeline.PipelineExecutionId}\","
                                "\"type\":\"PLAINTEXT\"},"
                            " {\"name\":\"TRAINED_MODEL_S3\","
                                "\"value\":\"#{CreateMLWorkflow.TRAINED_MODEL_S3}\","
                                "\"type\":\"PLAINTEXT\"},"
                            " {\"name\":\"INFERENCE_IMAGE_URI\","
                                "\"value\":\"#{CreateMLWorkflow.INFERENCE_IMAGE_URI}\","
                                "\"type\":\"PLAINTEXT\"},"
                            " {\"name\":\"TIMESTAMP\","
                                "\"value\":\"#{CreateMLWorkflow.TIMESTAMP}\","
                                "\"type\":\"PLAINTEXT\"},"
                            " {\"name\":\"EVAL_RESULT_PATH\","
                                "\"value\":\"#{CreateMLWorkflow.EVAL_RESULT_PATH}\","
                                "\"type\":\"PLAINTEXT\"}]"
                    },
                    'inputArtifacts': [
                        {
                            'name': 'Code'
                        },
                    ],
                    'namespace': 'Deploy'
                }],
        }],
    'version': 1
}

if codepipeline_name in get_pipeline_list():
    response = codepipeline_client.update_pipeline(
        pipeline=pipeline
    )
else:
    response = codepipeline_client.create_pipeline(
        pipeline=pipeline,
        tags=[
            {
                'key': 'owner',
                'value': user_name
            },
        ]
    )

CodePipeline のパイプラインが作成されました。パイプラインが自動的に開始しているはずですので、以下のセルを実行して表示されたリンクをクリックして CodePipeline のコンソールで様子を確認しましょう。

Source Stage -> Build Stage -> Approval Stage -> Deploy Stage の順でパイプラインが実行されます。Build Stage の実行が終わると、Approval Stage で手動承認待ちの状態になります。手動承認待ちの状態になったら、CodePipeline のコンソールに表示された [レビュー] ボタンをクリックして承認してください。引き続き、パイプラインで学習したモデルをデプロイするための Deploy Stage が実行されます。

Build Stage の実行には、CreateMLWorkflow と RunMLWorkflow にそれぞれ 20分ほどかかります。手動承認で承認したあとの Deploy Stage は 1分ほどで終わりますが、そのあと推論エンドポイントが InService の状態になるまでには数分かかります。

In [ ]:
from IPython.display import display, Markdown
display(Markdown(f"<a href=\"https://{region}.console.aws.amazon.com/codesuite/codepipeline/pipelines?region={region}\" target=\"_blank\">CodePipeline のコンソール</a>"))

## 記録された各種データの確認

### Amazon Experiments の中を確認

CodePipeline の実行が完了したら、Experiment の中をのぞいてみましょう。まずは Experiment のデータを ExperimentAnalytics を使って読み出します。なお、このサンプルでは Step Functions で作成した ML パイプラインモデル評価ジョブの中で Experiments にデータを登録しています。

In [ ]:
# search_expression = {
#     "Filters":[
#         {
#             "Name": "TrialComponentName",
#             "Operator": "Contains",
#             "Value": evaluation_job_name,
#         }
#     ],
# }

trial_component_analytics = ExperimentAnalytics(
    experiment_name=experiment_evaluate.experiment_name,
    sort_by="parameters.accuracy",
#     search_expression=search_expression,
#     sort_by="metrics.acc.max",
#     sort_order="Ascending",# Ascending or Descending
#     metric_names=['metric1', 'metric2'],
#     parameter_names=['accuracy', 'roc_auc'],
    input_artifact_names=[]
)

In [ ]:
import pandas as pd
df = trial_component_analytics.dataframe()
pd.set_option('display.max_columns', None)
df

### Model Registry の中を確認

以下のセルを実行して、簡易的にモデルのバージョンなどを確認します。このサンプルでは、手動承認されたモデルのみが記録されています。手動承認されたモデルだけではなく、CI/CD パイプラインで作成したすべてのモデルを Model Registry に登録したい場合は、学習ジョブか評価ジョブの中で `model.register`（deploy.py の `register_model()` 参照） を実行してモデルパッケージを登録し、deploy.py では [`update_model_package` API を使って](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-approve.html) モデルパッケージのステータスを Approved に変更してからモデルをデプロイしてください。

Amazon SageMaker Studio からは、Model Registry に登録されたモデルの精度などを確認することができます。

In [ ]:
import pprint

model_package_list = get_model_packages(model_package_group_name)

pprint.pprint(model_package_list)
model_package_list = [{'ModelPackageArn': r['ModelPackageArn'],
                       'CreationTime': r['CreationTime'].astimezone(JST).strftime('%Y/%m/%d-%H:%M:%S'),
                       'ModelPackageVersion': r['ModelPackageVersion'],
                       'ModelApprovalStatus': r['ModelApprovalStatus']} for r in model_package_list] 

特定のモデルバージョンの詳細情報を見たい場合は `describe_model_package` を使用します。Model Registry の UI か API でモデルを承認する際にコメントを入れておくと、`ApprovalDescription` のところに記録されます。

In [ ]:
response =sagemaker_client.describe_model_package(
        ModelPackageName=model_package_list[0]['ModelPackageArn']
    )
response

## 推論エンドポイントの動作確認

CodePipeline が Deploy Stage まで完了し、推論エンドポイントが InService になったら動作確認をしてみましょう。MNIST データセットをダウンロードし、推論エンドポイントに推論リクエストを投げます。

In [ ]:
while True:
    response = sagemaker_client.describe_endpoint(
        EndpointName= mlworkflow_name
    )
    if response['EndpointStatus'] == 'InService':
        break
    sleep(30)

In [ ]:
!aws s3 cp s3://fast-ai-imageclas/mnist_png.tgz . --no-sign-request
!tar -xvzf  mnist_png.tgz

In [ ]:
%matplotlib inline
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os
import random
import numpy as np
import matplotlib.pyplot as plt

test_dir = 'mnist_png/testing'

test_data = datasets.ImageFolder(root=test_dir,
                            transform=transforms.Compose([
                            transforms.Grayscale(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))

raw_test_data = datasets.ImageFolder(root=test_dir,
                                        transform=transforms.Compose([
                                        transforms.Grayscale(),
                                        transforms.ToTensor()]))
num_samples = 5
indices = random.sample(range(len(raw_test_data) - 1), num_samples)
raw_images = np.array([raw_test_data[i][0].numpy() for i in indices])
raw_labels = np.array([raw_test_data[i][1] for i in indices])


for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(raw_images[i].reshape(28, 28), cmap='gray')
    plt.title(raw_labels[i])
    plt.axis('off')
    
images = np.array([test_data[i][0].numpy() for i in indices])

In [ ]:
from sagemaker.pytorch.model import PyTorchPredictor

predictor = PyTorchPredictor(
        endpoint_name= mlworkflow_name
)

In [ ]:
prediction = predictor.predict(images)
predicted_label = prediction.argmax(axis=1)

print('The GT labels are: {}'.format(raw_labels))
print('The predicted labels are: {}'.format(predicted_label))

## [おまけ] Model Registry のモデルパッケージをデプロイ

以下のセルのコメントアウトを外してから実行すると、Model Registry に登録されたモデルパッケージを推論エンドポイントにデプロイすることができます。なお、モデルパッケージはステータスが Approved になっていないとデプロイできません。このサンプルでは、モデルパッケージのデプロイはモデルデプロイ用 CodeBuild プロジェクトが実行しています。

モデルパッケージをエンドポイントにデプロイすると、Model Registry に記録されているモデルパッケージの Activity 情報に ModelDeployment というイベントが追加されます。

In [ ]:
# model_package_arn = model_package_list[0]['ModelPackageArn']
# tmp_endpoint_name = model_package_group_name + '-tmp'
# model_package = sagemaker.model.ModelPackage(role=role,
#                              model_package_arn=model_package_arn)
# model_package.deploy(instance_type='ml.m5.xlarge',
#                      initial_instance_count=1,
#                      endpoint_name=tmp_endpoint_name)
# package_predictor = PyTorchPredictor( endpoint_name= tmp_endpoint_name)

# prediction = package_predictor.predict(images)
# predicted_label = prediction.argmax(axis=1)

# print('The GT labels are: {}'.format(raw_labels))
# print('The predicted labels are: {}'.format(predicted_label))
# package_predictor.delete_endpoint()

## リソースの削除

このノートブックの実行が終わったら、不要なリソースを削除することを忘れないでください。このノートブックを最後まで実行してリソースの削除をしたら、ノートブックインスタンス、各種データを保存した S3 バケットも不要であれば削除してください。

### Amazon SageMaker 推論エンドポイントの削除

In [ ]:
from sagemaker.pytorch.model import PyTorchPredictor

predictor = PyTorchPredictor(
        endpoint_name=mlworkflow_name
)
predictor.delete_endpoint()

### CodeCommit, CodeBuild, CodePipeline の削除

In [ ]:
response = codecommit_client.delete_repository(
    repositoryName=code_repository_name
)
print('Delete:', response['repositoryId'])

response = codebuild_client.delete_project(
    name=codebuild_project_name
)
print('Delete:', codebuild_project_name)

response = codebuild_client.delete_project(
    name=codebuild_deploy_project_name
)
print('Delete:', codebuild_deploy_project_name)

response = codepipeline_client.delete_pipeline(
    name=codepipeline_name
)
print('Delete:', codepipeline_name)

### Amazon ECR リポジトリの削除

In [ ]:
container_image_list = [
    project_name + '-base-' + user_name,
    project_name + '-lambda-' + user_name,
    project_name + '-preprocess-' + user_name,
    project_name + '-train-' + user_name,
    project_name + '-evaluate-' + user_name,
]
for i in container_image_list:
    try:
        ecr_client.delete_repository(
            repositoryName=i,
            force=True
        )
        print('Delete:', i)
    except Exception as e:
        print(e)
        pass

### Lambda 関数の削除

In [ ]:
lambda_function_list = list(set(lambda_function_list))
for f in lambda_function_list:
    lambda_client.delete_function(FunctionName=f)

In [ ]:
lambda_function_list

### Amazon SageMaker Experiment の削除

In [ ]:
sm = boto3.Session().client('sagemaker')
def cleanup(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(sagemaker_boto_client=sm, trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                sagemaker_boto_client=sm,
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            sleep(.5)
        trial.delete()
    experiment.delete()
    print('Delete:', experiment)
cleanup(experiment_evaluate)

### Amazon SageMaker Model Registry の削除

In [ ]:
model_package_list = get_model_packages(model_package_group_name)
model_package_list = [{'ModelPackageArn': r['ModelPackageArn'],
                       'CreationTime': r['CreationTime'].astimezone(JST).strftime('%Y/%m/%d-%H:%M:%S'),
                       'ModelPackageVersion': r['ModelPackageVersion'],
                       'ModelApprovalStatus': r['ModelApprovalStatus']} for r in model_package_list] 

for m in model_package_list:
    response = sagemaker_client.delete_model_package(
        ModelPackageName=m['ModelPackageArn']
    )

response = sagemaker_client.delete_model_package_group(
    ModelPackageGroupName=model_package_group_name
)

### AWS Step Functions ワークフローの削除

In [ ]:
workflow_list = Workflow.list_workflows()
workflow_arn = [d['stateMachineArn'] for d in workflow_list  if d['name']==mlworkflow_name][0]
sfn_workflow = Workflow.attach(workflow_arn)
try:
    sfn_workflow.delete()
    print('Delete:', mlworkflow_name)
except Exception as e:
    print(e)

### IAM Role と Policy の削除

In [ ]:
role_name_list = list(set(role_name_list))
policy_arn_list = list(set(policy_arn_list))

response = iam_client.detach_role_policy(
    RoleName=role.split('/')[2],
    PolicyArn=sagemaker_policy_arn
)


for r in role_name_list:
    try:
        detach_role_policies(r)
        iam_client.delete_role(RoleName=r)
        print('Delete:', r)
    except Exception as e:
        print(e)
        continue

for p in policy_arn_list:
    try:
        iam_client.delete_policy(PolicyArn=p)
        print('Delete:', p)
    except Exception as e:
        print(e)
        continue

### Amazon S3 バケットの削除
S3 バケットを削除したい場合は、以下のセルのコメントアウトを外してから実行してバケットを空にしてください。その後、コンソールからバケットの削除を実行してください。

In [ ]:
# def delete_all_keys_v2(bucket, prefix, dryrun=False):
#     contents_count = 0
#     next_token = ''

#     while True:
#         if next_token == '':
#             response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
#         else:
#             response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix, ContinuationToken=next_token)

#         if 'Contents' in response:
#             contents = response['Contents']
#             contents_count = contents_count + len(contents)
#             for content in contents:
#                 if not dryrun:
#                     print("Deleting: s3://" + bucket + "/" + content['Key'])
#                     s3_client.delete_object(Bucket=bucket, Key=content['Key'])
#                 else:
#                     print("DryRun: s3://" + bucket + "/" + content['Key'])

#         if 'NextContinuationToken' in response:
#             next_token = response['NextContinuationToken']
#         else:
#             break

#     print(contents_count, 'file were deleted.')

# delete_all_keys_v2(bucket_name, '')

## [Option] CI/CD パイプラインの定期実行

### flow.yml 更新用 Lambda 関数の作成
たとえば、最新のデータが s3://bucket/yyyymmdd のようなパスに格納されているとします。1日に1回、前日のデータを使ってパイプラインを実行したい場合に、以下の流れでパイプラインを実行することができます。

- Amazon EventBridge でイベントを発生させて Lambda 関数を実行
- Lambda 関数がdatetime を使ってデータ保存パスを生成して flow.yml に反映し CodeCommit に push
- CodeCommit へのファイル push をトリガに CodePipeline のパイプラインを実行

**なお、以下の EventBridge Rule 作成部分までコードを実行すると 10 分ごとに CodePipeline のパイプラインが実行されるようになりますが、前処理ジョブの入力に存在しないパス(timestampで自動的に作成されたパス)が設定されることになるため、Step Functions ワークフローで前処理ジョブを起動する部分で必ずエラーになります。**

パイプラインの定期実行方法の例としてご参照ください。

In [ ]:
lambda_event_function_name  = project_name + '-event-' + user_name
filepath = lambda_event_function_name + '.py'

flow_yml={f"""import yaml
import boto3
import json
from datetime import datetime
from dateutil import tz

codecommit_client = boto3.client('codecommit')

def lambda_handler(event, context):
    JST = tz.gettz('Asia/Tokyo')
    timestamp = datetime.now(JST).strftime('%Y%m%d-%H%M%S')
    print(timestamp)
    

    file_name = 'flow.yml'
    response = codecommit_client.get_file(
        repositoryName='{code_repository_name}',
        filePath=file_name
    )

    commit_id = response['commitId']
    file = response['fileContent']

    flow = yaml.safe_load(file.decode())
    flow['preprocess']['input-data-path'] = f's3://{bucket_name}/data/{{timestamp}}'

    out_file = f'/tmp/{{file_name}}'
    with open(out_file, 'w') as f:
        yaml.dump(flow, f)

    with open(out_file, 'r') as f:
        response = codecommit_client.put_file(
            repositoryName='{code_repository_name}',
            branchName='main',
            fileContent=f.read(),
            filePath=file_name,
            parentCommitId=commit_id,
            commitMessage='push from lambda',
            name='system',
            email='test@amazon.co.jp'
        )

    return {{
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }}
"""
}

with open(filepath, 'w') as f:
    f.write('\n'.join(list(flow_yml)))

In [ ]:
lambda_event_policy_name = lambda_event_function_name + '-policy'
lambda_event_role_name = lambda_event_function_name + '-role'

inline_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            'Effect': 'Allow',
            'Action': 'logs:CreateLogGroup',
            'Resource': f'arn:aws:logs:{region}:{account_id}:*'
        },
        {
            'Effect': 'Allow',
            'Action': [
                'logs:CreateLogStream',
                'logs:PutLogEvents'
            ],
            'Resource': [
                f'arn:aws:logs:{region}:{account_id}:log-group:/aws/lambda/{lambda_event_function_name}:*'
            ]
        }
    ]
}

assume_role_policy = {
  "Version": "2012-10-17",
  "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"lambda.amazonaws.com"},"Action": "sts:AssumeRole"}]
}

policy_list=['AWSCodeCommitPowerUser']

create_policy_role(lambda_event_policy_name, inline_policy,
                   lambda_event_role_name, assume_role_policy, policy_list=policy_list)
lambda_event_role_arn = iam_client.get_role(RoleName=lambda_event_role_name)['Role']['Arn']
sleep(10) # wait until IAM is created

以下のセルでは、Lambda 関数で使用するライブラリとソースコードを zip に固めています。Lmabda 関数を作成する際は、以下の処理を実行した環境と同じ Python のバージョンのランタイムを指定してください。

In [ ]:
%%sh -s $lambda_event_function_name

rm -rf $1
rm $1.zip
mkdir $1
pip install pyyaml -t $1
cp $1.py $1
cd $1
zip -r ../$1.zip .

In [ ]:
lambda_event_function_arn = create_lambda_function(lambda_event_function_name,
                                                   lambda_event_function_name,
                                                   lambda_event_role_arn,
                                                   py_version='python3.6')

### 定期実行のための Amazon EventBridge Rule 作成

このサンプルでは、イベントを発生させるためにのイベントバスを使用しています。定期イベントを発生させるために、EventBridge のコンソールの左側のメニューにある [イベントバス] をクリックし、デフォルトのイベントバス部分の [アクション]-> [検出を開始する] をクリックしてください。[スキーマの検出] 列に {Started] と記載されていることを確認してください。

以下のセルでは、10 分ごとにイベントが発生するようルールを作成しています。

In [ ]:
eventbridge_client = boto3.client('events', region_name=region)
start_pipeline_rule_name = project_name + '-StartPipeline-' + user_name

response = eventbridge_client.put_rule(
    Name=start_pipeline_rule_name,
    ScheduleExpression='rate(10 minutes)',
    State='ENABLED',
    Description='schedule to start pipeline',
)
start_pipeline_rule_arn = response['RuleArn']

以下のセルでは、作成したルールに Lambda 関数を紐づけています。

In [ ]:
function_arn = lambda_client.get_function(FunctionName=lambda_event_function_name)['Configuration']['FunctionArn']
response = eventbridge_client.put_targets(
    Rule=start_pipeline_rule_name,
    Targets=[
        {
            'Id': lambda_event_function_name,
            'Arn': function_arn,
        }
    ]
)

response = lambda_client.add_permission(
    FunctionName=lambda_event_function_name,
    StatementId=lambda_event_function_name,
    Action='lambda:InvokeFunction',
    Principal='events.amazonaws.com',
    SourceArn=start_pipeline_rule_arn,
    SourceAccount=account_id,
)

### リソースの削除
定期実行を止めるために、必ず以下のセルを実行してください。

In [ ]:
# Amazon EventBridge Rule の削除
response = eventbridge_client.remove_targets(
    Rule=start_pipeline_rule_name,
    Ids=[
        lambda_event_function_name,
    ],
    Force=True
)

response = eventbridge_client.delete_rule(
    Name=start_pipeline_rule_name,
    Force=True
)

In [ ]:
# データパス作成用 Lambda 関数の削除
lambda_client.delete_function(FunctionName=lambda_event_function_name)
lambda_function_list.remove(lambda_event_function_name)